In [15]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import numpy as np

#data path 수정
os.chdir("C:\\Users\\Owner2\\Desktop\\G팔로미_vuno")
os.getcwd()

'C:\\Users\\Owner2\\Desktop\\G팔로미_vuno'

In [16]:
train_dir = './train_withoout_callback'

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    #featurewise_center=True, #center 중심 유지
    #featurewise_std_normalization=True, #normalization
    rotation_range=25.0,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #brightness_range=[0.2,1.0],
    zoom_range=[0.8,1.2],
    horizontal_flip=True,
    validation_split=0.2) #수평방향 뒤집기

In [18]:
train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지의 크기를 150 × 150로 변경합니다
        target_size=(224,224),
        batch_size=32,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(224,224),
    batch_size=32,
    subset='validation') # set as validation data

Found 1372 images belonging to 4 classes.
Found 340 images belonging to 4 classes.


In [19]:
from keras import models, layers
from keras import Input
from keras.applications import VGG16
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))
model.trainable = True
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
#customizing my layers
additional_model = models.Sequential()
additional_model.add(model)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(1024, activation='relu'))
additional_model.add(layers.Dense(256, activation='relu'))
additional_model.add(layers.Dense(4, activation='softmax'))

additional_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_8 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 1028      
Total params: 40,669,252
Trainable params: 40,669,252
Non-trainable params: 0
_________________________________________________________________


In [21]:
def f1score(precision, recall):
    _f1score = ( 2 * recall * precision) / (recall + precision)
    
    # return a single tensor value
    return _f1score

In [22]:
# checkpoint = ModelCheckpoint(filepath='My_VGG_{epoch:03d}_{val_loss:.7f}.hdf5',monitor='loss', mode='min', save_best_only=True)
checkpoint = ModelCheckpoint(filepath='VGG16_png_1.hdf5', 
            monitor='val_loss', 
            mode='min', 
            save_best_only=True)

In [23]:
import tensorflow as tf

additional_model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc',tf.keras.metrics.AUC(),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

In [24]:
class_weight = {0: 1.1,
                1: 1.1,
                2: 1,
                3: 1}

In [25]:
history = additional_model.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples /32,
      epochs=60,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples/32,
      callbacks=[checkpoint],
      class_weight=class_weight)

Epoch 1/60
43/42 [==============================] - 18s 419ms/step - loss: 1.4514 - acc: 0.2602 - auc_1: 0.4991 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - f1score: 0.1000 - val_loss: 1.3847 - val_acc: 0.2706 - val_auc_1: 0.5023 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_f1score: 0.1000
Epoch 2/60
43/42 [==============================] - 17s 388ms/step - loss: 1.4511 - acc: 0.2711 - auc_1: 0.5046 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - f1score: 0.1000 - val_loss: 1.3863 - val_acc: 0.2706 - val_auc_1: 0.5066 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_f1score: 0.1000
Epoch 3/60
43/42 [==============================] - 17s 391ms/step - loss: 1.4511 - acc: 0.2711 - auc_1: 0.5076 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - f1score: 0.1000 - val_loss: 1.3857 - val_acc: 0.2706 - val_auc_1: 0.5078 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_f1score: 0.1000
Epoch 4/60
43/42 [==============================] - 17s

KeyboardInterrupt: 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(1, len(acc) + 1)
 
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.figure()
 
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Loss')
plt.legend()
 
plt.show()

In [ ]:
_loss, _acc, _auc, _precision, _recall, _f1score =  additional_model.evaluate_generator(validation_generator, steps=32)
print('loss: {:.3f}, accuracy: {:.3f}, auc: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _auc, _precision, _recall, _f1score))

In [ ]:
from keras.models import Model, load_model

new_model = models.load_model('VGG16_png_1.hdf5',compile=False)


new_model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc',tf.keras.metrics.AUC(),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

In [ ]:
test_dir='./test_withoout_callback'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32)

In [ ]:
_loss, _acc, _auc, _precision, _recall, _f1score =new_model.evaluate_generator(test_generator, steps=len(test_generator.samples)/32)
print('loss: {:.3f}, accuracy: {:.3f}, auc: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _auc, _precision, _recall, _f1score))